# Task 1: Does increasing the bitrate (independent variable) from 2000 kbps to 6000 kbps for the first game using a resolution of 1080p and a framerate of 60 fps (conditions 36 and 50) significantly improve the video quality ratings (dependent variable). Use the ratings provided in the gaming video quality dataset. In case of an extreme outlier, the participant can be completely removed from the dataset.

### Step 1: Include libraries

In [1]:
# install.packages('dplyr')      # processing 
# install.packages('gdata')      # file reading
# install.packages('effsize')      # Cohen's D

In [2]:
library(dplyr)     # processing
library(readxl)    # reading in data
library(effsize)   # Cohen's D

Warning message:
"package 'dplyr' was built under R version 3.6.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'effsize' was built under R version 3.6.2"

### Step 2: Load data sets

In [3]:
# read in separate data sets for 2000 & 6000 kbps
get_gaming_data_for_kbps <- function(kbps) {
    gaming_data <- read_excel("datasets/DB01_gaming_video_quality_dataset.xlsx")
    gaming_data <- gaming_data %>% filter(Resolution == 1080, 
                                      Framerate == 60, 
                                      Game == 'Game1', 
                                      (Condition == 36 | Condition == 50),
                                      Bitrate == kbps) %>% 
                                    select(PID, VQ) %>%
                                    arrange(PID)
    
    if(kbps == 2000) {
        gaming_data <- gaming_data %>% rename(VQ_2000 = VQ)
    } else {
        gaming_data <- gaming_data %>% rename(VQ_6000 = VQ)
    }
    
}

head(get_gaming_data_for_kbps(2000))
head(get_gaming_data_for_kbps(6000))

PID,VQ_2000
1,2.9
2,4.6
3,2.7
4,2.6
5,2.7
6,3.0


PID,VQ_6000
1,4.5
2,5.7
3,4.5
4,3.5
5,4.5
6,5.0


In [4]:
# merge separate data sets for 2000 & 6000 kbps by PID
gaming_data <- cbind(get_gaming_data_for_kbps(2000), get_gaming_data_for_kbps(6000))[-c(3)]
head(gaming_data)

PID,VQ_2000,VQ_6000
1,2.9,4.5
2,4.6,5.7
3,2.7,4.5
4,2.6,3.5
5,2.7,4.5
6,3.0,5.0


### Step 3: Decide on which test to use for the research question

### As the exact same individuals [see PID] were asked to rate the video quality several times => dependent t-test / repeated measure t-test / Paired Samples t-test to be used here!

### Step 4: Check if conditions for Paired Samples / dependent t-test are met

#### 1.) Dependent variable [= VQ] measured at least at interval / ration level => OK, as VQ is continuous 
#### 2.) No significant outliers => Don't know, need to check that in the next step!
#### 3.) Independent variable [= kbps] should consist of two categorical related groups => OK, as 2000kbps & 6000kbps [= related] and also categorical [always either 2000kbps or 6000kbps]
#### 4.) Distribution of differences in dependent variable [= VQ] between two related groups approximately normally distributed => Don't know, need to check in the next step!

### Step 5: Check not yet confirmed requirements => no significant outliers in data set & differences approximately normally distributed?

In [5]:
# Check for outliers using Z-Score method
get_significant_outliers <- function(data) {
    data %>% 
    mutate(Std_Dev_VQ_2000 = sd(VQ_2000), 
           Std_Dev_VQ_6000 = sd(VQ_6000), 
           Mean_VQ_2000 = mean(VQ_2000), 
           Mean_VQ_6000 = mean(VQ_6000)) %>%
    mutate(Z_Score_2000 = (VQ_2000 - Mean_VQ_2000) / Std_Dev_VQ_2000) %>%
    mutate(Z_Score_6000 = (VQ_6000 - Mean_VQ_6000) / Std_Dev_VQ_6000) %>%
    select(PID, VQ_2000, VQ_6000, Z_Score_2000, Z_Score_6000) %>% 
    filter(abs(Z_Score_2000) > 3.29 | abs(Z_Score_6000) > 3.29)
}

get_significant_outliers(gaming_data)

PID,VQ_2000,VQ_6000,Z_Score_2000,Z_Score_6000


#### No significant outliers found neither in 2000kbps nor in 6000kbps data => condition 2.) met!

In [6]:
# Compute pairwise differences in VQ ratings
VQ_differences <- (gaming_data %>% mutate(VQ_Differences = as.numeric(VQ_6000 - VQ_2000)))
head(VQ_differences)

# Conduct Shapiro-Wilk-Test & just to be sure also Kolmogorov-Smirnov test for normality check
shapiro.test(VQ_differences[['VQ_Differences']])
ks.test(VQ_differences[['VQ_Differences']], "pnorm", mean=mean(VQ_differences[['VQ_Differences']]), sd=sd(VQ_differences[['VQ_Differences']]))

PID,VQ_2000,VQ_6000,VQ_Differences
1,2.9,4.5,1.6
2,4.6,5.7,1.1
3,2.7,4.5,1.8
4,2.6,3.5,0.9
5,2.7,4.5,1.8
6,3.0,5.0,2.0



	Shapiro-Wilk normality test

data:  VQ_differences[["VQ_Differences"]]
W = 0.9397, p-value = 0.1457


Warning message in ks.test(VQ_differences[["VQ_Differences"]], "pnorm", mean = mean(VQ_differences[["VQ_Differences"]]), :
"ties should not be present for the Kolmogorov-Smirnov test"


	One-sample Kolmogorov-Smirnov test

data:  VQ_differences[["VQ_Differences"]]
D = 0.13542, p-value = 0.749
alternative hypothesis: two-sided


#### As p-values of both the Kolmogrov-Smirnov-Test & Shapiro-Wilk-Test >= 0.05 => normal distribution of VQ rating differences can be assumed, hence condition 4.) met!

#### All the requirements for the Paired Sample t-test are met! :)

### Step 6: Conduct Paired Sample / dependent t-test

#### We wish to check if the mean of the 6000kbps distribution is greater than the one of the 2000kbps distribution. In other words, we want to check if the difference of the 6000kbps mean and the 2000kbps mean [call that difference m] is greater than 0. 

#### Hence:H0: m <= 0, H1: m > 0

In [7]:
# Conduct Paired Sample t-test
t.test(gaming_data[['VQ_6000']], gaming_data[['VQ_2000']], paired=TRUE, alternative = "greater")


	Paired t-test

data:  gaming_data[["VQ_6000"]] and gaming_data[["VQ_2000"]]
t = 6.7225, df = 24, p-value = 2.966e-07
alternative hypothesis: true difference in means is greater than 0
95 percent confidence interval:
 0.9810747       Inf
sample estimates:
mean of the differences 
                  1.316 


#### As p-value < 0.05 => null hypothesis rejected => H1 (= increasing bitrate from 2.000 to 6.000 kbps improves video quality ratings) assumed

### Step 7: Compute effect size

In [8]:
# Compute Cohen's D
paste0('Already implemented Cohens d function:')
cohen.d(gaming_data[['VQ_6000']], gaming_data[['VQ_2000']], paired=TRUE)

compute_cohens_d_alternatively <- function(data) {
    distribution_parameters <- (data %>% mutate(Mean_VQ_2000 = mean(VQ_2000), 
                                Mean_VQ_6000 = mean(VQ_6000), 
                                Sd_VQ_2000 = sd(VQ_2000),
                                Sd_VQ_6000 = sd(VQ_6000)))[1,]
    mean_6000 <- distribution_parameters[['Mean_VQ_6000']]
    mean_2000 <- distribution_parameters[['Mean_VQ_2000']]
    sd_6000 <- distribution_parameters[['Sd_VQ_6000']]
    sd_2000 <- distribution_parameters[['Sd_VQ_2000']]
    
    return(round((mean_6000 - mean_2000) / sqrt((sd_2000 * sd_2000 + sd_6000 * sd_6000) / 2), digits=3))
    
}

paste0('Self-implemented Cohens d function => Effect size = ', compute_cohens_d_alternatively(gaming_data))

[1] "Already implemented Cohens d function:"


Cohen's d

d estimate: 1.51107 (large)
95 percent confidence interval:
    lower     upper 
0.8496679 2.1724725 

[1] "Self-implemented Cohens d function => Effect size = 1.512"

### Step 8: Interpretation of results

#### With a p-value of 2.966e-07 / a t-score (test statistic value) of 6.7225, n-1=25-1=24 degrees of freedom & a Cohen's d effect size of around 1.51 [= large], we were able to show with a statistical significance of alpha = 0.05 that the average difference of 6000kbps VQ ratings & 2000kbps VQ ratings between the two dependent samples (each of size n=25) is greater than 0: increasing the bitrate from 2.000 to 6.000 kbps significantly improves the video quality ratings (VQ) here.